## Proximity to urban center (≥ 20,000 people) from charcoal production centroids

In [2]:
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape, Point
from shapely.ops import nearest_points
import geopandas as gpd
import numpy as np
import pandas as pd

In [4]:
zambia_fp = '/home/h99g576/zambia/Data/district.shp'
zambia_gdf = gpd.read_file(zambia_fp)
zambia_gdf.set_crs(epsg=4326, inplace=True)
zambia_gdf = zambia_gdf.rename(columns={'NAME_2': 'district'})
zambia_gdf['district'] = zambia_gdf['district'].str.strip().str.title()

# load population raster
raster_path = 'Data/zmb_ppp_2020_UNadj.tif'
with rasterio.open(raster_path) as src:
    pop = src.read(1)
    transform = src.transform
    crs = src.crs

# mask out cells with low population
pop_masked = np.where(pop >= 1, pop, 0)
mask = pop_masked > 0

# extract polygons from nonzero population pixels
polygons = []
pops = []

for geom, val in shapes(pop_masked.astype('float32'), mask=mask, transform=transform):
    if val > 0:
        polygons.append(shape(geom))
        pops.append(val)

gdf_pop = gpd.GeoDataFrame({'population': pops}, geometry=polygons, crs=crs)
gdf_pop['dummy'] = 1


clusters = gdf_pop.dissolve(by='dummy', aggfunc='sum').explode(index_parts=False).reset_index(drop=True)
urban_clusters = clusters[clusters['population'] >= 20000].copy()
urban_clusters_utm = urban_clusters.to_crs(epsg=32735)
urban_clusters_utm['centroid'] = urban_clusters_utm.geometry.centroid
urban_centroids = urban_clusters_utm.set_geometry('centroid')

# get district centroids
zambia_gdf_utm = zambia_gdf.to_crs(epsg=32735)

# calculate distance
def nearest_distance_polygon(row, urban_geom):
    nearest_geom = urban_geom.unary_union
    nearest_point = nearest_points(row.geometry, nearest_geom)[1]
    return row.geometry.distance(nearest_point)

districts_utm = zambia_gdf.to_crs(epsg=32735)
districts_utm['min_dist_to_urban_km'] = districts_utm.apply(
    nearest_distance_polygon, axis=1, urban_geom=urban_clusters_utm
) / 1000

dist_summary = districts_utm[['district', 'min_dist_to_urban_km']].copy()

# export to csv
dist_summary.to_csv('urban_proximity.csv', index=False)

In [3]:
# group by district and calculate mean/min distance
dist_summary = charcoal_points_utm.groupby('district')['dist_to_urban_km'].min().reset_index()
dist_summary.rename(columns={'dist_to_urban_km': 'min_dist_to_urban_km'}, inplace=True)

# export to csv
dist_summary.to_csv('min_distance_urban_km.csv', index=False)

In [9]:
print(charcoal_points_utm)

    district  charcoal  longitude   latitude                         geometry  \
0     Mumbwa      18.0  26.544014 -14.967912   POINT (450970.357 8345172.708)   
1    Masaiti      51.0  28.649769 -13.387429   POINT (678643.806 8519423.670)   
2    Mpongwe      25.0  27.742530 -13.580970   POINT (580331.595 8498493.302)   
3    Lundazi       3.0  33.237824 -12.398848  POINT (1179256.358 8621379.464)   
4    Petauke      11.0  31.343907 -14.073172   POINT (969352.405 8439848.846)   
5   Mufumbwe      14.0  25.228858 -13.826156   POINT (308562.952 8470792.589)   
6    Solwezi      23.0  26.504885 -12.560471   POINT (446215.543 8611418.802)   
7      Mbala       9.0  31.427956  -8.963148   POINT (987208.190 9006284.608)   
8     Mungwi      34.0  31.736708  -9.892888  POINT (1019852.288 8902732.044)   
9      Choma      26.0  26.986164 -16.762439   POINT (498525.384 8146724.812)   
10   Namwala      21.0  26.774275 -15.941772   POINT (475842.246 8237492.432)   

    dist_to_urban_km  
0   